<a href="https://colab.research.google.com/github/avalenciacu/VALENCIA-ALEXIS-s-y-s/blob/main/youtube_detector_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#cargar datos desde drive acceso libre
#1UVHl8wyAafw1kgvke9IMi7fwUNv8ZswA
FILEID = "18Ul8ZQSXMWoeoNrmb5fD1vYQ9vU5bxGA" #"1DxI5wQpqEWksw2BqJnG7n0IgNQ_xByX-"

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O  canciones.xlsx && rm -rf /tmp/cookies.txt
#!unzip -o codigos.zip
!dir

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_ = 'canciones.xlsx'#leer archivo xlsx con link, band, type
X  = pd.read_excel(file_)
X#imprimir filas iniciales

In [ ]:
#instalar librerias necesarias para descargar audios youtube
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile

# Paquete de carga de cookies para el control anti-bots captcha
!pip install browser-cookie3

import os
import yt_dlp as youtube_dl
import browser_cookie3

try:
    cookies = browser_cookie3.firefox()
except:
    print("No se pueden descargar cookies desde firefox. Intentando Chrome...")
    try:
        cookies = browser_cookie3.chrome()
    except:
        print("No se pueden descargar cookies desde Chrome. Por favor asegúrate de estar logueado en Youtube desde tu navegador.")
        cookies = None

#funcion para descargar mp3 desde youtube
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':f'{name}.mp3',
    }
    if cookies:
        options['cookiefile'] = None
        options['cookiejar'] = cookies;

    with youtube_dl.YoutubeDL(options) as ydl:
        try:
            video_info = ydl.extract_info(video_url, download=False)
            ydl.download([video_info['webpage_url']])
            print("Download complete... {}".format(filename))
        except Exception as e:
            print(f"Error descargando {video_url}: {e}")

In [ ]:
import subprocess
import os

#crear carpeta con resultados
try:
  os.mkdir('results')
except:
  print("Carpeta results ya existe")

#recorrer excel con videos
N, P = X.shape
Ns = N * 5 #cantidad de segmentos por cancion

for n in range(N):
    print(f"video {n+1} de {N}")
    print(f"link: {X.loc[n,'link']}\n")
    print(f"band: {X.loc[n,'band']}\n")
    print(f"type: {X.loc[n,'type']}\n")
    #ruta video n-th
    name_ = 'results/'+X.loc[n,'band']+"_"+str(n)+"_"+str(X.loc[n,'type_num']) # #video+nombre+tipo de genero musical
    #descargar mp3 desde youtube
    try: # Added try block to catch download errors
        download_ytvid_as_mp3(X.loc[n,'link'],name_)
        #convertir a .wav
        subprocess.call(['ffmpeg','-y', '-i', name_+'.mp3',
                       name_+'.wav'])
    except youtube_dl.utils.DownloadError as e: # Catch the specific DownloadError
        print(f"Skipping video {X.loc[n,'link']} due to download error: {e}") # Print a message and skip

In [ ]:
#cargar .wavs y partir audios
#lista archivos .wav
path = 'results/'
wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]
wav_files

In [ ]:
import soundfile as sf # para instalar pip install soundfile
from scipy.signal import resample_poly
import numpy as np
import os

#leer archivos y crear np.array audios
fs = 48000
tl = np.array([30,40,50,60,70,80]) #puntos lectura
ts = 5 #t segmento
# The total number of samples for each segment at the target fs
segment_length_fs = int(ts * fs)
Ns = len(wav_files)*len(tl) #cantidad segmentos
# x_t should be initialized with the shape corresponding to the target fs
x_t = np.zeros((Ns, segment_length_fs, 2)) #Ns segmentos, cantidad de muestras, 2 canales (stereo)
label = np.zeros((Ns,1)) #vector tipo de genero
type_c = X['type'].unique()
name_c = []
#leer archivos wav
i = 0
for name in wav_files:#lectura audio .wav
    x, fs_i = sf.read(path+name)
    # Ensure x is at least 2D (for stereo)

    if x.ndim == 1:
        x = np.expand_dims(x, axis=1) # Add a channel dimension if mono
    if x.shape[1] == 1: # If still mono after expand_dims
        x = np.repeat(x, 2, axis=1) # Convert mono to stereo by repeating the channel

    # Check if the audio file has enough data for the requested segments
    if len(x) < int(fs_i * (tl[-1] + ts)):
        print(f"Warning: File {name} is too short for all requested segments. Skipping some segments.")
        valid_tl = tl[tl + ts <= len(x)/fs_i]
    else:
        valid_tl = tl

    for ti in valid_tl: #segmentos de tiempo
        print(name,'x',x.shape,'fs actual:',fs_i)

        # Slice based on the original sampling rate fs_i
        start_sample = int(fs_i * ti)
        end_sample = int(fs_i * (ti + ts))
        xc = x[start_sample:end_sample, :]

        # Check if the sliced segment has enough data for resampling
        # This can happen if the calculated end_sample exceeds the actual audio length
        if xc.shape[0] < int(ts * fs_i):
            print(f"Warning: Segment from {ti}s to {ti+ts}s in {name} is shorter than expected. Skipping.")
            continue # Skip this segment

        if fs_i != fs:
            # Resample using resample_poly
            # The output length of resample_poly is len(xc) * up / down
            # We want the resampled length to be exactly segment_length_fs
            # So we need to calculate up and down such that len(xc) * up / down = segment_length_fs
            # Or simply use the desired number of samples directly if supported by resample_poly
            # resample_poly can take 'nyq' or 'kaiser_beta' arguments for filter design,
            # but for simple resampling to a target number of points, a direct approach is needed.
            # A simpler resample function might be better, or calculate up/down differently.

            # Let's use a method that targets the desired number of samples directly.
            # scipy.signal.resample can do this, but it uses FFT and might not be ideal.
            # Let's adjust the resampling calculation with resample_poly to ensure the correct output size.
            # Target output size is segment_length_fs.
            # Current size is xc.shape[0].
            # We want xc.shape[0] * (up / down) = segment_length_fs
            # Let's find up and down that approximate fs/fs_i
            # A more robust way is to use a library that handles resampling to a specific length.
            # For now, let's ensure the resampled array is truncated or padded if necessary,
            # although it's better to get the resampling right.

            # A common approach for arbitrary resampling is
            # import librosa
            # xc_resampled = librosa.resample(xc, orig_sr=fs_i, target_sr=fs)
            # However, this outputs a 1D array for mono. Need to handle stereo.

            # Let's stick with resample_poly but ensure the output is the correct size.
            # If resampling makes it slightly longer or shorter, we'll handle it.
            gcd_val = np.gcd(fs, fs_i)
            up_val = fs // gcd_val
            down_val = fs_i // gcd_val

            # Resample each channel separately
            xc_resampled_ch1 = resample_poly(xc[:, 0], up=up_val, down=down_val)
            xc_resampled_ch2 = resample_poly(xc[:, 1], up=up_val, down=down_val)

            # Combine channels
            xc_resampled = np.stack((xc_resampled_ch1, xc_resampled_ch2), axis=-1)

            # Ensure the resampled segment has the target length (segment_length_fs)
            if xc_resampled.shape[0] > segment_length_fs:
                xc_resampled = xc_resampled[:segment_length_fs, :]
            elif xc_resampled.shape[0] < segment_length_fs:
                # Pad with zeros if shorter
                padding = np.zeros((segment_length_fs - xc_resampled.shape[0], 2))
                xc_resampled = np.vstack((xc_resampled, padding))

            xc = xc_resampled # Use the resampled array
            print(f"Resampled to shape: {xc.shape}")

        # Now xc should have the shape (segment_length_fs, 2) which is (240000, 2)
        if xc.shape[0] != segment_length_fs:
             print(f"Error: Resampled segment has incorrect length {xc.shape[0]}. Expected {segment_length_fs}. Skipping.")
             continue # Skip this segment if resampling failed to produce correct length


        x_t[i] = xc
        # The label indexing seems to assume the order of wav_files matches the order in X
        # and the type_num is the last digit before '.wav' and is 1 or 2.
        # This might be fragile. Consider using the 'type_num' column from X directly
        # based on the filename or a mapping.
        # For now, sticking to the original logic for the label.
        try:
             # Extract type_num from the filename
             parts = name.split('_')
             type_num_str = parts[-1].split('.')[0]
             label[i] = int(type_num_str)
        except (IndexError, ValueError):
             print(f"Warning: Could not extract type_num from filename {name}. Setting label to 0.")
             label[i] = 0 # Assign a default or error label

        # The name_c logic also seems to assume the filename format.
        # Sticking to original logic.
        name_c += [name[:-4]] # Remove '.wav' from the end


        print(f"{i} lectura: {name}; segundo {ti}:{ti+ts}; tipo música: {type_c[int(label[i])-1]}")
        i+=1

# After the loop, if some segments were skipped, the actual number of populated segments in x_t and label
# might be less than Ns. We should truncate these arrays to the actual number of populated segments.
# Ns was calculated based on the initial assumption of processing all segments.
# Let's re-calculate the effective Ns (number of processed segments).
effective_Ns = i
x_t = x_t[:effective_Ns]
label = label[:effective_Ns]
name_c = name_c[:effective_Ns]

x_t.shape

In [ ]:
from IPython.display import Audio #reproducir segmento
i = 0
Audio(x_t[i].T,rate=fs)

In [ ]:
#calculo de fourier
vf = np.fft.rfftfreq(x_t.shape[1],1/fs) #calculo vector de frecuencias
Xw = np.fft.rfft(x_t,axis=1).mean(axis=-1) #transformada rapida de Fourier para señal Real a lo largo del tiempo (axis=1) y se promedian los dos canales
Xw.shape

In [ ]:
#grafica tiempo y fourier
plt.plot(np.arange(0,ts,1/fs),x_t.mean(axis=-1).T) #se promedian los dos canales stereo
plt.xlabel('t[s]')
plt.ylabel('x(t)')
plt.show()

In [ ]:
plt.plot(vf,abs(Xw).T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

In [ ]:
#se normalizan espectros entre 0 y 1 para evitar inconsistencias por ampliltudes máximas
from sklearn.preprocessing import MinMaxScaler
sca = MinMaxScaler()
Xw_ = sca.fit_transform(abs(Xw).T).T

plt.plot(vf,Xw_.T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

#en dB
plt.plot(vf,(20*np.log10(Xw_+1e-10)).T) # se suma 1e-10 para evitar discontinuidad del log
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)| dB')
plt.show()

**Nota**: Generalmente el espectro se presenta en [decibeles [dB]](https://es.wikipedia.org/wiki/Decibelio)

## Visualización de los datos en 2D

In [ ]:
from umap import UMAP
#visualización de datos
n_neighbors = int(2*np.sqrt(Xw_.shape[0]))
sca_ = MinMaxScaler()

red_ = UMAP(n_components=2,n_neighbors=n_neighbors)
X_2D = sca_.fit_transform(red_.fit_transform(Xw_))

In [ ]:
#graficar separabilidad 2D
plt.scatter(X_2D[:,0],X_2D[:,1],c=label)
plt.colorbar()
plt.show()

color_ = ["b","y"]
#nombre cancion
plt.scatter(X_2D[:,0],X_2D[:,1],c=label,s=1)
for i, tex in enumerate(name_c):
    #print(f"{i} {tex}")
    plt.text(X_2D[i,0]*1.025,X_2D[i,1]*1.025, tex[:-2]+"_"+str(i), fontsize=6,color=color_[int(label[i]-1)])

plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.show()

In [ ]:
#reproducir audio
i = 1
Audio(x_t[i].T,rate=fs)

# Predicción del género musical sobre espectro de Fourier

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Crear una instancia del clasificador RandomForest
model = KNeighborsClassifier(n_neighbors=5)

# Realizar la validación cruzada
# Xw_ son tus características y label son tus etiquetas
# cv define el número folds para la validación cruzada
scores = cross_val_score(model, Xw_, label.ravel(), cv=5,scoring='accuracy')

# Imprimir los resultados de la validación cruzada
print("Acierto para cada Fold:", scores)
print("Acierto promedio:", scores.mean())
print("Desviación estándar del Acierto:", scores.std())

#entrenar modelo para predecir nuevas canciones
model.fit(Xw_, label.ravel())

In [ ]:
import joblib
#guardar modelo
try:
  os.mkdir('modelo')
except:
  print("Carpeta modelo ya existe")

filename_ = 'modelo/reggaeton_vs_metal'
model_ ={'Xw_':Xw_, 'label' : label, 'name_c' : name_c, 'vf':vf,'fs':fs, 'modelo':model,'type':X['type'].unique()}
joblib.dump(model_,filename_+".pkl")


In [ ]:
#descargar modelo
#guardar resultados
from google.colab import files
from datetime import date, datetime
import shutil
#guardar resultados
namefile = str(datetime.now().strftime("%Y_%m_%d_%H_%M_%d"))+'modelo'
shutil.make_archive(namefile, 'zip', 'modelo')
files.download(namefile+'.zip')

#el archivo .zip puede cargarse en drive y utilizarse en otro cuaderno para detectar género musical de nuevos segmentos

# Para una nueva canción

In [ ]:
#cargar modelo
my_model_loaded = joblib.load(filename_+".pkl")
my_model_loaded.keys()

In [ ]:
#evaluar para un segmento en fft normalizado

pred_ = my_model_loaded['modelo'].predict(Xw_[0].reshape(1,-1))[0]
print('Etiqueta estimada: ',my_model_loaded['type'][int(pred_-1)])
print('Etiqueta orignal: ',my_model_loaded['type'][int(my_model_loaded['label'][0][0])-1])

# **ESTREAMLIT**

In [ ]:
!pip install streamlit joblib yt-dlp soundfile scipy umap-learn pandas numpy matplotlib

In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import yt_dlp as youtube_dl
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import resample_poly
from sklearn.preprocessing import MinMaxScaler
from umap import UMAP # Although UMAP is not strictly needed for the prediction itself, it was used in the original notebook for visualization and might be useful later or for debugging.
import subprocess
import time # Import time for potential delays

# Load the trained model and related data
# Make sure the 'modelo' directory and 'reggaeton_vs_metal.pkl' file exist
# in the same directory where you run this script, or provide the correct path.
try:
    my_model_loaded = joblib.load('modelo/reggaeton_vs_metal.pkl')
    model = my_model_loaded['modelo']
    vf = my_model_loaded['vf']
    fs = my_model_loaded['fs']
    type_c = my_model_loaded['type']
    # Xw_ and label are not strictly needed for prediction on a new sample,
    # but keeping them here in case they are used for visualization or other purposes later.
    # Xw_ = my_model_loaded['Xw_']
    # label = my_model_loaded['label']
    # st.success("Modelo cargado exitosamente.") # Avoid Streamlit calls outside st functions
except FileNotFoundError:
    # st.error("Error: Archivo del modelo 'modelo/reggaeton_vs_metal.pkl' no encontrado. Asegúrate de que la carpeta 'modelo' y el archivo .pkl estén en la ubicación correcta.")
    st.stop() # Stop the app if the model cannot be loaded.
except Exception as e:
    # st.error(f"Error al cargar el modelo: {e}")
    st.stop()


# Define the audio processing parameters
ts = 5  # segment time in seconds
segment_length_fs = int(ts * fs)

# Function to download audio as mp3 from YouTube (adapted from your original code)
def download_ytvid_as_mp3_streamlit(video_url, output_path):
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':f'{output_path}.mp3',
        'nocheckcertificate':True, # Added to potentially help with certificate issues
        'no_warnings':True, # Suppress warnings
        'quiet':True, # Suppress verbose output
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }],
         'extract_audio': True,
    }
    try:
        with youtube_dl.YoutubeDL(options) as ydl:
            ydl.download([video_url])
        return True, None # Return True on success
    except Exception as e:
        return False, e # Return False and the error on failure


# Function to process the downloaded audio (adapted from your original code)
def process_audio_for_prediction(audio_path, target_fs, segment_duration):
    try:
        x, fs_i = sf.read(audio_path)

        if x.ndim == 1:
            x = np.expand_dims(x, axis=1)
        if x.shape[1] == 1:
            x = np.repeat(x, 2, axis=1)

        # Take a segment for analysis (e.g., from 30s to 35s as in your original notebook)
        # You might want to make this starting point configurable or analyze multiple segments
        start_time = 30 # seconds
        start_sample = int(fs_i * start_time)
        end_sample = int(fs_i * (start_time + segment_duration))

        if end_sample > len(x):
            st.warning(f"El audio es demasiado corto para tomar el segmento desde {start_time}s. Tomando el último segmento posible.")
            end_sample = len(x)
            start_sample = int(end_sample - fs_i * segment_duration)
            if start_sample < 0:
                 start_sample = 0 # Ensure start_sample is not negative
                 st.warning("El audio es demasiado corto incluso para un segmento completo. Analizando el audio disponible.")

        xc = x[start_sample:end_sample, :]

        if fs_i != target_fs:
            # Resample using resample_poly
            gcd_val = np.gcd(target_fs, fs_i)
            up_val = target_fs // gcd_val
            down_val = fs_i // gcd_val

            # Ensure that the resampled signal has the target length
            target_length = int(segment_duration * target_fs)
            xc_resampled_ch1 = resample_poly(xc[:, 0], up=up_val, down=down_val, N=target_length)
            xc_resampled_ch2 = resample_poly(xc[:, 1], up=up_val, down=down_val, N=target_length)

            xc_resampled = np.stack((xc_resampled_ch1, xc_resampled_ch2), axis=-1)
            xc = xc_resampled

        # Calculate Fourier Transform
        # Ensure the segment has the correct length before FFT
        if xc.shape[0] != int(segment_duration * target_fs):
             st.error(f"Error de procesamiento: el segmento de audio no tiene la longitud esperada para FFT. Esperado: {int(segment_duration * target_fs)}, Obtenido: {xc.shape[0]}")
             return None, "Longitud de segmento incorrecta después del remuestreo."


        Xw = np.fft.rfft(xc, axis=0).mean(axis=-1) # FFT along time axis (axis=0), then average channels

        # Normalize spectrum
        sca = MinMaxScaler()
        # Need to reshape for MinMaxScaler as it expects a 2D array
        Xw_ = sca.fit_transform(np.abs(Xw).reshape(-1, 1)).T # Reshape and transpose to match original Xw_ shape

        # Ensure the normalized spectrum has the correct number of features
        expected_features = len(vf) # Use the vf from the loaded model
        if Xw_.shape[1] != expected_features:
             st.error(f"Error de procesamiento: las características extraídas no tienen el tamaño esperado. Esperado: {expected_features}, Obtenido: {Xw_.shape[1]}")
             return None, "Número de características extraídas incorrecto."

        return Xw_, None
    except Exception as e:
        return None, e


# Streamlit App
st.title("Detector de Género Musical (Rock vs Reggaeton)")

st.write("Pega el enlace de un video de YouTube y el modelo intentará predecir si el género musical es Rock o Reggaeton.")

youtube_url = st.text_input("Enlace de YouTube:")

# Add a placeholder for status messages
status_text = st.empty()

if st.button("Analizar"):
    if youtube_url:
        status_text.info(f"Analizando: {youtube_url}")

        # Define a temporary file name for the downloaded audio
        temp_audio_mp3 = "temp_audio.mp3"
        temp_audio_wav = "temp_audio.wav"

        # Clean up previous temporary files if they exist
        if os.path.exists(temp_audio_mp3):
            os.remove(temp_audio_mp3)
        if os.path.exists(temp_audio_wav):
            os.remove(temp_audio_wav)

        # 1. Download the audio
        status_text.text("Descargando audio...")
        success, error = download_ytvid_as_mp3_streamlit(youtube_url, temp_audio_mp3[:-4]) # Pass name without extension

        if not success:
            status_text.error(f"Error al descargar el video: {error}")
        else:
            status_text.success("Audio descargado.")
            status_text.text("Convirtiendo a WAV...")
            # 2. Convert to WAV
            try:
                # Use absolute paths for subprocess for better reliability
                temp_audio_mp3_abs = os.path.abspath(temp_audio_mp3)
                temp_audio_wav_abs = os.path.abspath(temp_audio_wav)

                subprocess.run(['ffmpeg', '-y', '-i', temp_audio_mp3_abs, temp_audio_wav_abs],
                               check=True, # Raise an exception if the command fails
                               capture_output=True, # Capture stdout and stderr
                               text=True) # Decode output as text

                status_text.success("Conversión a WAV completada.")

                # 3. Process the audio and get features
                status_text.text("Procesando audio y extrayendo características...")
                Xw_, error = process_audio_for_prediction(temp_audio_wav_abs, fs, ts)

                if error:
                    status_text.error(f"Error al procesar el audio: {error}")
                elif Xw_ is None:
                     status_text.error("Error al procesar el audio: no se pudieron extraer características.")
                else:
                    status_text.success("Procesamiento completado.")

                    # 4. Make a prediction
                    status_text.text("Realizando predicción...")
                    try:
                        pred_ = model.predict(Xw_)[0]
                        # Ensure the predicted label index is within the bounds of type_c
                        if 0 <= int(pred_-1) < len(type_c):
                             predicted_genre = type_c[int(pred_-1)]
                             st.subheader("Resultado de la Predicción:")
                             st.write(f"El género musical predicho es: **{predicted_genre}**")
                        else:
                             status_text.error(f"Error: El modelo predijo una etiqueta inesperada: {pred_}")


                    except Exception as e:
                        status_text.error(f"Error al realizar la predicción: {e}")

            except FileNotFoundError:
                 status_text.error("Error: FFmpeg no encontrado. Asegúrate de que FFmpeg esté instalado y en el PATH.")
            except subprocess.CalledProcessError as e:
                 status_text.error(f"Error durante la conversión con FFmpeg: {e.stderr}")
            except Exception as e:
                status_text.error(f"Error durante la conversión o procesamiento: {e}")
            finally:
                # Clean up temporary files
                if os.path.exists(temp_audio_mp3):
                    os.remove(temp_audio_mp3)
                if os.path.exists(temp_audio_wav):
                    os.remove(temp_audio_wav)
                status_text.text("Limpieza de archivos temporales completada.")
    else:
        status_text.warning("Por favor, introduce un enlace de YouTube.")

st.markdown("""
---
Este detector utiliza un modelo de clasificación entrenado con espectros de Fourier de segmentos de audio.
""")

In [ ]:
# Install ngrok
!pip install ngrok

# Run the Streamlit app in the background
# Use nohup to allow the process to continue after the Colab cell finishes execution (though the tunnel will close)
# Redirect output to prevent it from cluttering the Colab output
!nohup streamlit run app.py &>/dev/null&

# Allow some time for the Streamlit app to start
import time
time.sleep(5)

# Start ngrok and capture the public URL
from pyngrok import ngrok

# Kill any running ngrok processes to avoid conflicts
ngrok.kill()

# Start a new ngrok tunnel to the Streamlit port (default is 8501)
# Get the public URL
try:
    public_url = ngrok.connect(addr="8501", proto="http").public_url
    print(f"Tu aplicación Streamlit está disponible en: {public_url}")
except Exception as e:
    print(f"Error al iniciar ngrok: {e}")
    print("Por favor, intenta ejecutar la celda nuevamente.")

He generado dos celdas de código:

1.  La primera celda utiliza `%%writefile app.py` para guardar el código de la aplicación Streamlit en un archivo llamado `app.py`. He realizado algunas modificaciones al código anterior para que funcione mejor en un entorno de servidor como Streamlit (por ejemplo, usando `st.info`, `st.success`, `st.error` para los mensajes de estado en lugar de `print`). También he ajustado la función `process_audio_for_prediction` para manejar el remuestreo y la verificación de la longitud del segmento de manera más robusta.
2.  La segunda celda instala `ngrok`, ejecuta la aplicación Streamlit en segundo plano usando `nohup` y luego utiliza `pyngrok` para crear un túnel HTTP a la aplicación Streamlit que se ejecuta en el puerto 8501 (el puerto por defecto de Streamlit).

Una vez que ejecutes la segunda celda, deberías ver un mensaje que te proporciona la URL pública donde puedes acceder a tu detector de género musical.

**Importante:** El túnel `ngrok` y la aplicación Streamlit se detendrán cuando la sesión de Colab termine o se desconecte el entorno de ejecución. Si quieres que la aplicación esté disponible por más tiempo, necesitarías desplegarla en un servicio de hosting dedicado para Streamlit o una plataforma en la nube.

¡Espero que esto te sea útil! Ejecuta las celdas y prueba tu detector de música.